<a href="https://colab.research.google.com/github/SATVIK202004/CodeAlpha_Music-Generation-with-AI/blob/main/CodeAlpha_MUSIC_GENERATION_WITH_AI_TASK3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install music21

In [11]:
from google.colab import drive

In [12]:
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import os
import numpy as np
import tensorflow as tf
from google.colab import drive
import librosa
from IPython.display import Audio

# Mount Google Drive
drive.mount('/content/drive')

# Path to the audio file
file_path = '/content/drive/My Drive/[iSongs.info] 06 - Sarrainodu.mp3'

# Load and play original audio file
audio_data, sr = librosa.load(file_path, sr=None)
print("Original audio:")
Audio(data=audio_data, rate=sr)

# Feature extraction
hop_length = 512
n_fft = 2048
n_mfcc = 13

mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc).T

# Normalize MFCCs
mfccs_mean = np.mean(mfccs, axis=0)
mfccs_std = np.std(mfccs, axis=0)
mfccs_normalized = (mfccs - mfccs_mean) / mfccs_std

# Define sequences and targets
sequence_length = 100
X = []
y = []
for i in range(len(mfccs_normalized) - sequence_length):
    X.append(mfccs_normalized[i:i+sequence_length])
    y.append(mfccs_normalized[i+sequence_length])

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Define RNN model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(sequence_length, n_mfcc)),
    tf.keras.layers.Dense(n_mfcc)
])

model.compile(loss='mse', optimizer='adam')

# Train the model
model.fit(X, y, epochs=10, batch_size=64, validation_split=0.2)

# Generate music sequence
seed_sequence = X[0]
generated_sequence = []

for i in range(1000):  # Adjust the length of the generated sequence as needed
    # Predict the next step
    next_step = model.predict(seed_sequence.reshape(1, sequence_length, n_mfcc))
    generated_sequence.append(next_step)
    # Update seed sequence for next prediction
    seed_sequence = np.concatenate((seed_sequence[1:], next_step), axis=0)

# Convert generated sequence back to audio
generated_audio = librosa.feature.inverse.mfcc_to_audio(np.array(generated_sequence).squeeze().T)

# Play the generated audio
print("Generated audio:")
Audio(data=generated_audio, rate=sr)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Original audio:
Epoch 1/10
317/317 [==============================] - 5s 9ms/step - loss: 0.2466 - val_loss: 0.1156
Epoch 2/10
317/317 [==============================] - 3s 10ms/step - loss: 0.1115 - val_loss: 0.1005
Epoch 3/10
317/317 [==============================] - 3s 8ms/step - loss: 0.1040 - val_loss: 0.0972
Epoch 4/10
317/317 [==============================] - 2s 8ms/step - loss: 0.1011 - val_loss: 0.0979
Epoch 5/10
317/317 [==============================] - 2s 8ms/step - loss: 0.0996 - val_loss: 0.0953
Epoch 6/10
317/317 [==============================] - 2s 8ms/step - loss: 0.0983 - val_loss: 0.0958
Epoch 7/10
317/317 [==============================] - 3s 9ms/step - loss: 0.0971 - val_loss: 0.0940
Epoch 8/10
317/317 [==============================] - 3s 9ms/step - loss: 0.0961 - val_loss: 0.0952
Epoch 9/10
317/317 [==============================] - 